<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

     |████████████████████████████████| 92kB 3.6MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Autosaving every 120 seconds


In [2]:
data_with_all_tags = pd.read_csv("/content/drive/My Drive/ML/data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,absurd,action,adult comedy,allegory,alternate history,alternate reality,anti war,atmospheric,autobiographical,avant garde,blaxploitation,bleak,boring,brainwashing,christian film,claustrophobic,clever,comedy,comic,cruelty,cult,cute,dark,depressing,dramatic,entertaining,fantasy,feel-good,flashback,good versus evil,...,grindhouse film,haunting,historical,historical fiction,home movie,horror,humor,insanity,inspiring,intrigue,magical realism,melodrama,murder,mystery,neo noir,non fiction,paranormal,philosophical,plot twist,pornographic,prank,psychedelic,psychological,queer,realism,revenge,romantic,sadist,satire,sci-fi,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [0]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [0]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [0]:
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize,  binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [6]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,1))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 657) Y : (10989, 71)
Dimensions of test data X: (2768, 657) Y: (2768, 71)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [7]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [8]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.1165, recall1: 0.6461, F1-measure: 0.1974


In [9]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  The Woods Are Full of Cuckoos
Actual genre:  psychedelic
Predicted tag:  ['absurd' 'adult comedy' 'alternate reality' 'boring' 'christian film'
 'clever' 'comedy' 'cute' 'entertaining' 'fantasy' 'feel-good' 'gothic'
 'home movie' 'humor' 'inspiring' 'paranormal' 'philosophical'
 'pornographic' 'prank' 'psychedelic' 'queer' 'romantic' 'satire'
 'whimsical'] 

Movie:  Good Luck Chuck
Actual genre:  comedy
Predicted tag:  ['adult comedy' 'bleak' 'comedy' 'cute' 'depressing' 'dramatic'
 'entertaining' 'feel-good' 'home movie' 'humor' 'melodrama'
 'pornographic' 'prank' 'psychological' 'queer' 'romantic' 'satire'
 'storytelling'] 

Movie:  Demonic Toys
Actual genre:  violence, humor
Predicted tag:  ['cruelty' 'cult' 'good versus evil' 'horror' 'murder' 'revenge' 'sadist'
 'sci-fi' 'suspenseful' 'violence'] 

Movie:  Behind the Candelabra
Actual genre:  queer, flashback
Predicted tag:  ['boring' 'comedy' 'dark' 'depressing' 'dramatic' 'entertaining'
 'flashback' 'melodrama' 'philosop

<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [10]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [11]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.1596, recall2: 0.5875, F1-measure: 0.2511


In [12]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  This Is Spinal Tap
Actual genre:  comedy, boring, cult, absurd, humor, satire, psychedelic
Predicted tag:  ['adult comedy' 'boring' 'brainwashing' 'clever' 'comedy' 'cult' 'cute'
 'entertaining' 'humor' 'prank' 'psychedelic' 'satire' 'stupid'
 'thought-provoking'] 

Movie:  Neerja
Actual genre:  tragedy
Predicted tag:  ['alternate history' 'comedy' 'historical' 'home movie' 'humor' 'murder'
 'satire' 'stupid'] 

Movie:  Anomalisa
Actual genre:  absurd
Predicted tag:  ['adult comedy' 'bleak' 'boring' 'comedy' 'cult' 'dark' 'dramatic'
 'feel-good' 'flashback' 'home movie' 'intrigue' 'mystery' 'neo noir'
 'philosophical' 'plot twist' 'psychedelic' 'psychological' 'queer'
 'satire' 'sentimental' 'storytelling' 'suicidal' 'thought-provoking'
 'tragedy'] 

Movie:  Undead
Actual genre:  comedy, murder, bleak, cult, violence, flashback
Predicted tag:  ['allegory' 'atmospheric' 'cult' 'murder' 'realism' 'suspenseful'
 'violence'] 

Movie:  Persuasion
Actual genre:  romantic
Predicted ta

<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [13]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [14]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.1515, recall3: 0.5868, F1-measure: 0.2408


In [15]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  Van Helsing: The London Assignment
Actual genre:  murder
Predicted tag:  ['alternate reality' 'blaxploitation' 'cruelty' 'cult' 'good versus evil'
 'murder' 'psychedelic' 'sci-fi'] 

Movie:  Offret
Actual genre:  insanity, dramatic, philosophical
Predicted tag:  ['alternate reality' 'avant garde' 'bleak' 'haunting' 'historical fiction'
 'melodrama' 'plot twist' 'psychedelic' 'sci-fi' 'sentimental'
 'storytelling'] 

Movie:  Forever Knight
Actual genre:  flashback
Predicted tag:  ['alternate reality' 'comedy' 'good versus evil' 'paranormal' 'plot twist'
 'realism' 'sci-fi' 'tragedy' 'violence'] 

Movie:  Anastasia
Actual genre:  romantic, cute, entertaining
Predicted tag:  ['allegory' 'alternate reality' 'bleak' 'boring' 'brainwashing' 'comedy'
 'cult' 'depressing' 'entertaining' 'fantasy' 'feel-good' 'flashback'
 'good versus evil' 'gothic' 'humor' 'paranormal' 'psychedelic' 'satire'
 'stupid' 'whimsical'] 

Movie:  The Raven
Actual genre:  murder
Predicted tag:  ['atmospheric'

<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [16]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [17]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.1667, recall4: 0.5836, F1-measure: 0.2593


In [18]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

Movie:  Walking Tall Part II
Actual genre:  revenge, murder, violence
Predicted tag:  ['action' 'boring' 'brainwashing' 'clever' 'comedy' 'cult' 'flashback'
 'humor' 'murder' 'revenge' 'stupid' 'suspenseful' 'violence'] 

Movie:  Our Family Wedding
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Doragon bôru Z: Sûpâ senshi gekiha! Katsu no wa ore da
Actual genre:  violence
Predicted tag:  ['cult' 'fantasy' 'good versus evil' 'revenge' 'romantic' 'sci-fi'
 'tragedy' 'violence'] 

Movie:  Aftermath
Actual genre:  violence
Predicted tag:  ['atmospheric' 'dark' 'flashback' 'historical' 'horror' 'murder' 'realism'
 'revenge' 'sadist' 'satire' 'suspenseful' 'tragedy' 'violence'] 

Movie:  Equus
Actual genre:  insanity, flashback
Predicted tag:  ['atmospheric' 'cult' 'fantasy' 'flashback' 'gothic' 'philosophical'
 'plot twist' 'psychedelic' 'psychological' 'queer' 'satire' 'sentimental'
 'thought-provoking'] 



<h1>2. TfidfVectorizer with (1 - 2 Grams):<?h1>

In [19]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,2))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 71)
Dimensions of test data X: (2768, 666) Y: (2768, 71)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [20]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [21]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.1170, recall5: 0.6448, F1-measure: 0.1980


In [22]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

Movie:  Phantom
Actual genre:  psychedelic, atmospheric
Predicted tag:  ['absurd' 'action' 'alternate history' 'alternate reality' 'anti war'
 'boring' 'brainwashing' 'claustrophobic' 'comedy' 'comic' 'cult'
 'good versus evil' 'historical' 'historical fiction' 'humor' 'murder'
 'mystery' 'psychedelic' 'revenge' 'sci-fi' 'stupid' 'suspenseful'
 'thought-provoking' 'violence'] 

Movie:  Ida
Actual genre:  murder
Predicted tag:  ['atmospheric' 'boring' 'depressing' 'dramatic' 'flashback' 'haunting'
 'horror' 'insanity' 'melodrama' 'psychedelic' 'romantic' 'sentimental'
 'storytelling'] 

Movie:  Kaakha..Kaakha: The Police
Actual genre:  murder, violence, flashback, good versus evil, romantic, revenge
Predicted tag:  ['murder' 'neo noir' 'realism' 'revenge' 'sadist' 'tragedy' 'violence'] 

Movie:  Wong Fei Hung III: Si wong jaang ba
Actual genre:  cult, violence
Predicted tag:  ['absurd' 'action' 'avant garde' 'comic' 'cute' 'entertaining'
 'good versus evil' 'historical' 'historical fict

<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [23]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [24]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.1583, recall6: 0.5831, F1-measure: 0.2490


In [25]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

Movie:  The Tempest
Actual genre:  insanity, revenge, avant garde, psychedelic
Predicted tag:  ['fantasy' 'good versus evil' 'melodrama' 'romantic'] 

Movie:  The Royal Tenenbaums
Actual genre:  dark, comedy, bleak, flashback, psychedelic, romantic, entertaining
Predicted tag:  ['action' 'atmospheric' 'boring' 'dark' 'depressing' 'dramatic'
 'entertaining' 'flashback' 'home movie' 'melodrama' 'romantic'
 'sentimental' 'storytelling' 'thought-provoking' 'tragedy'] 

Movie:  The Sting
Actual genre:  boring, depressing, murder, cult, plot twist, clever, inspiring, revenge, entertaining
Predicted tag:  ['comedy' 'comic' 'cult' 'flashback' 'humor' 'murder' 'neo noir' 'prank'
 'revenge' 'satire'] 

Movie:  The Stöned Age
Actual genre:  cult
Predicted tag:  ['absurd' 'adult comedy' 'comedy' 'comic' 'cult' 'entertaining' 'humor'
 'intrigue' 'pornographic' 'prank' 'psychedelic' 'revenge' 'satire'
 'stupid'] 

Movie:  A Beautiful Mind
Actual genre:  suspenseful, psychological, dramatic, cult, fl

<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [26]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [27]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.1550, recall7: 0.5800, F1-measure: 0.2446


In [28]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

Movie:  Payback
Actual genre:  comedy, neo noir, violence, atmospheric, flashback, plot twist, clever, humor, action, revenge, sadist
Predicted tag:  ['absurd' 'action' 'alternate reality' 'atmospheric' 'bleak' 'boring'
 'comedy' 'cult' 'dark' 'depressing' 'dramatic' 'entertaining' 'flashback'
 'home movie' 'humor' 'insanity' 'murder' 'mystery' 'neo noir'
 'psychedelic' 'realism' 'revenge' 'sadist' 'storytelling' 'stupid'
 'suspenseful' 'violence'] 

Movie:  Along the Great Divide
Actual genre:  revenge
Predicted tag:  ['flashback' 'melodrama' 'murder' 'mystery' 'paranormal' 'revenge'
 'romantic' 'sadist' 'suspenseful' 'tragedy' 'violence'] 

Movie:  Way of the Vampire
Actual genre:  violence, flashback
Predicted tag:  ['cult' 'dark' 'good versus evil' 'gothic' 'horror' 'murder' 'realism'
 'revenge' 'sadist' 'tragedy' 'violence'] 

Movie:  Uno dopo l'altro
Actual genre:  western, violence
Predicted tag:  ['avant garde' 'claustrophobic' 'comedy' 'dark' 'dramatic' 'entertaining'
 'histor

<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [29]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [30]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.1673, recall8: 0.5839, F1-measure: 0.2601


In [31]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

Movie:  Dilwale Dulhania Le Jayenge
Actual genre:  romantic
Predicted tag:  ['cute' 'feel-good' 'flashback' 'melodrama' 'prank' 'realism' 'romantic'
 'sentimental' 'tragedy'] 

Movie:  Athadu
Actual genre:  murder, flashback
Predicted tag:  ['action' 'comedy' 'cruelty' 'cult' 'entertaining' 'flashback' 'murder'
 'mystery' 'neo noir' 'prank' 'realism' 'revenge' 'sadist' 'stupid'
 'suspenseful' 'violence'] 

Movie:  Fur: An Imaginary Portrait of Diane Arbus
Actual genre:  flashback, home movie
Predicted tag:  ['adult comedy' 'boring' 'flashback' 'mystery' 'romantic'] 

Movie:  Las brujas de Zugarramurdi
Actual genre:  romantic
Predicted tag:  ['alternate history' 'bleak' 'boring' 'cruelty' 'cult' 'paranormal'
 'violence'] 

Movie:  Body Snatchers
Actual genre:  cult, suspenseful, psychedelic, satire
Predicted tag:  ['action' 'alternate reality' 'fantasy' 'satire' 'sci-fi' 'sentimental'] 



<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [32]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,3))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 71)
Dimensions of test data X: (2768, 666) Y: (2768, 71)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [33]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [34]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.1170, recall9: 0.6448, F1-measure: 0.1980


In [35]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

Movie:  Monkey Shines
Actual genre:  revenge, murder
Predicted tag:  ['flashback' 'philosophical' 'psychedelic' 'romantic'] 

Movie:  Fingersmith
Actual genre:  historical fiction
Predicted tag:  ['alternate reality' 'atmospheric' 'bleak' 'boring' 'clever' 'comedy'
 'cruelty' 'cute' 'dark' 'depressing' 'dramatic' 'entertaining' 'fantasy'
 'feel-good' 'flashback' 'gothic' 'haunting' 'horror' 'humor' 'insanity'
 'melodrama' 'mystery' 'paranormal' 'plot twist' 'prank' 'psychedelic'
 'psychological' 'romantic' 'satire' 'sentimental' 'storytelling'
 'whimsical'] 

Movie:  The Ghost Writer
Actual genre:  boring, mystery, murder, dramatic, intrigue, atmospheric, suspenseful
Predicted tag:  ['absurd' 'action' 'allegory' 'alternate history' 'anti war' 'atmospheric'
 'autobiographical' 'avant garde' 'bleak' 'boring' 'brainwashing'
 'christian film' 'claustrophobic' 'clever' 'dark' 'depressing' 'dramatic'
 'flashback' 'gothic' 'haunting' 'historical' 'historical fiction'
 'home movie' 'insanity' 

<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [36]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [37]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.1553, recall10: 0.5728, F1-measure: 0.2444


In [38]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

Movie:  This Revolution
Actual genre:  romantic, avant garde, violence, flashback
Predicted tag:  ['blaxploitation' 'bleak' 'boring' 'murder' 'realism' 'suspenseful'] 

Movie:  Wonder Woman
Actual genre:  psychedelic, murder
Predicted tag:  ['avant garde' 'comedy' 'entertaining' 'historical' 'historical fiction'
 'humor' 'psychedelic' 'satire'] 

Movie:  The Petrified Forest
Actual genre:  depressing, murder
Predicted tag:  ['alternate reality' 'atmospheric' 'avant garde' 'bleak' 'boring' 'clever'
 'cult' 'dark' 'depressing' 'fantasy' 'good versus evil' 'humor'
 'plot twist' 'psychedelic' 'romantic' 'satire' 'storytelling'] 

Movie:  Deewane
Actual genre:  romantic
Predicted tag:  ['allegory' 'christian film' 'philosophical' 'romantic' 'tragedy'] 

Movie:  Worry Dolls
Actual genre:  murder
Predicted tag:  ['cruelty' 'historical' 'home movie' 'murder' 'neo noir' 'plot twist'
 'realism' 'revenge' 'sadist' 'violence'] 



<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [39]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [40]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.1565, recall11: 0.5784, F1-measure: 0.2464


In [41]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

Movie:  Jack the Giant Slayer
Actual genre:  revenge, fantasy, romantic, humor
Predicted tag:  ['absurd' 'action' 'avant garde' 'bleak' 'comedy' 'dark' 'entertaining'
 'fantasy' 'good versus evil' 'horror' 'insanity' 'psychedelic'
 'storytelling' 'violence'] 

Movie:  Annie
Actual genre:  depressing
Predicted tag:  ['clever' 'comedy' 'cute' 'dramatic' 'entertaining' 'fantasy' 'gothic'
 'home movie' 'humor' 'intrigue' 'melodrama' 'psychedelic' 'romantic'
 'sci-fi' 'storytelling'] 

Movie:  Bon appétit
Actual genre:  romantic
Predicted tag:  ['alternate reality' 'avant garde' 'fantasy' 'melodrama' 'philosophical'
 'plot twist' 'pornographic' 'romantic' 'thought-provoking'] 

Movie:  Cet obscur objet du désir
Actual genre:  whimsical, absurd, psychedelic, satire, romantic
Predicted tag:  ['atmospheric' 'avant garde' 'bleak' 'cruelty' 'dark' 'depressing'
 'entertaining' 'historical' 'humor' 'psychedelic' 'romantic' 'satire'
 'storytelling' 'stupid' 'violence'] 

Movie:  Tekken 5
Actual gen

<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [42]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

In [46]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

Movie:  Scooby-Doo Goes Hollywood
Actual genre:  psychedelic
Predicted tag:  ['absurd' 'comic' 'humor' 'psychedelic' 'satire'] 

Movie:  Force 10 from Navarone
Actual genre:  psychedelic
Predicted tag:  ['action' 'comedy' 'cult' 'humor' 'mystery' 'prank' 'sadist' 'suspenseful'
 'violence'] 

Movie:  Long-Distance Princess
Actual genre:  romantic
Predicted tag:  ['prank' 'romantic' 'sentimental'] 

Movie:  Original Sin
Actual genre:  suspenseful, boring, murder, violence, flashback
Predicted tag:  ['atmospheric' 'bleak' 'flashback' 'humor' 'intrigue' 'murder'
 'plot twist' 'romantic' 'tragedy' 'whimsical'] 

Movie:  Kaze no na wa amunejia
Actual genre:  psychedelic, violence, romantic
Predicted tag:  ['alternate reality' 'avant garde' 'cult' 'fantasy' 'gothic' 'insanity'
 'psychedelic' 'sci-fi' 'violence'] 



<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [47]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 71)
Dimensions of test data X: (2768, 666) Y: (2768, 71)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [48]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [49]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.1170, recall13: 0.6448, F1-measure: 0.1980


In [50]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

Movie:  Shottas
Actual genre:  violence
Predicted tag:  ['blaxploitation' 'comedy' 'cult' 'flashback' 'murder' 'neo noir'
 'realism' 'revenge' 'sadist' 'suspenseful' 'tragedy' 'violence'] 

Movie:  Duplicity
Actual genre:  violence, intrigue, romantic, flashback
Predicted tag:  ['absurd' 'adult comedy' 'alternate reality' 'bleak' 'boring'
 'claustrophobic' 'clever' 'comedy' 'cute' 'dark' 'depressing' 'dramatic'
 'entertaining' 'feel-good' 'flashback' 'home movie' 'humor' 'inspiring'
 'intrigue' 'mystery' 'neo noir' 'philosophical' 'plot twist' 'prank'
 'psychological' 'queer' 'realism' 'romantic' 'satire' 'sentimental'
 'storytelling' 'stupid' 'suicidal' 'suspenseful' 'thought-provoking'
 'whimsical'] 

Movie:  Let Us Prey
Actual genre:  violence, murder, flashback
Predicted tag:  ['atmospheric' 'boring' 'claustrophobic' 'clever' 'comedy' 'cruelty'
 'cult' 'dark' 'flashback' 'gothic' 'grindhouse film' 'haunting' 'horror'
 'insanity' 'murder' 'mystery' 'neo noir' 'paranormal' 'plot twis

<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [51]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [52]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.1606, recall14: 0.5947, F1-measure: 0.2529


In [53]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

Movie:  2010
Actual genre:  cult, boring
Predicted tag:  ['alternate history' 'alternate reality' 'atmospheric' 'cult'
 'psychedelic' 'sci-fi' 'sentimental' 'suspenseful'] 

Movie:  Revenge of the Zombies
Actual genre:  revenge
Predicted tag:  ['comedy' 'gothic' 'sci-fi'] 

Movie:  Largo Winch II
Actual genre:  flashback
Predicted tag:  ['absurd' 'action' 'bleak' 'comedy' 'entertaining' 'intrigue' 'murder'
 'neo noir' 'revenge' 'sadist' 'satire' 'suspenseful' 'violence'] 

Movie:  Bolt
Actual genre:  comedy, alternate reality, cute
Predicted tag:  ['absurd' 'alternate reality' 'boring' 'comedy' 'comic' 'entertaining'
 'humor' 'psychedelic' 'romantic' 'satire'] 

Movie:  The Sum of Us
Actual genre:  flashback
Predicted tag:  ['cute' 'intrigue' 'romantic' 'sentimental'] 



<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [54]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [55]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.1556, recall15: 0.5869, F1-measure: 0.2460


In [56]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

Movie:  Provoked
Actual genre:  paranormal
Predicted tag:  ['cruelty' 'historical' 'insanity' 'intrigue' 'melodrama' 'murder'
 'psychological' 'revenge' 'sadist' 'satire'] 

Movie:  Independence Day
Actual genre:  mystery, stupid, cult, humor, action, tragedy, suspenseful
Predicted tag:  ['action' 'alternate history' 'alternate reality' 'atmospheric' 'comic'
 'cult' 'dark' 'good versus evil' 'gothic' 'humor' 'mystery' 'paranormal'
 'psychedelic' 'satire' 'sci-fi' 'suspenseful' 'tragedy' 'violence'] 

Movie:  Repentance
Actual genre:  horror, murder
Predicted tag:  ['allegory' 'flashback' 'historical' 'magical realism' 'psychedelic'
 'sadist'] 

Movie:  Deceived
Actual genre:  intrigue, neo noir
Predicted tag:  ['absurd' 'action' 'allegory' 'atmospheric' 'boring' 'claustrophobic'
 'comedy' 'depressing' 'dramatic' 'fantasy' 'flashback' 'gothic'
 'haunting' 'horror' 'insanity' 'murder' 'mystery' 'paranormal'
 'plot twist' 'psychedelic' 'revenge' 'suspenseful'] 

Movie:  The Texas Chainsaw

<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [57]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [58]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.1673, recall16: 0.5839, F1-measure: 0.2601


In [59]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

Movie:  The King of Comedy
Actual genre:  cult, humor, psychedelic, psychological
Predicted tag:  ['alternate history' 'alternate reality' 'boring' 'comedy' 'cult'
 'depressing' 'flashback' 'humor' 'neo noir' 'philosophical' 'queer'
 'revenge' 'romantic' 'suspenseful'] 

Movie:  The Family That Preys
Actual genre:  cruelty
Predicted tag:  ['boring' 'comedy' 'dark' 'depressing' 'dramatic' 'flashback' 'home movie'
 'humor' 'intrigue' 'neo noir' 'satire' 'storytelling'] 

Movie:  Real Life
Actual genre:  satire
Predicted tag:  ['autobiographical' 'bleak' 'comedy' 'cruelty' 'cult' 'historical fiction'
 'home movie' 'psychedelic' 'psychological' 'satire'] 

Movie:  Transylvania 6-5000
Actual genre:  humor
Predicted tag:  ['comedy' 'humor' 'inspiring' 'melodrama' 'murder' 'tragedy'] 

Movie:  The Gazebo
Actual genre:  suspenseful, murder
Predicted tag:  ['comedy' 'murder' 'neo noir' 'plot twist' 'sadist' 'suspenseful'] 



<h1> Conclusion: </h1>

In [60]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])



print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.116   | 0.646  |  0.197   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |    0.16   | 0.587  |  0.251   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |   0.151   | 0.587  |  0.241   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.167   | 0.584  |  0.259   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.117   | 0.645  |  0.198   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.158   | 0.583  |  0.249   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.155   |  0.58  |  0.

<h1>5. Word2Vec</h1>

In [61]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|██████████| 10989/10989 [00:00<00:00, 24708.03it/s]


In [0]:
with open('/content/drive/My Drive/ML/glove.6B.300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [63]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|██████████| 10989/10989 [00:07<00:00, 1569.01it/s]


In [64]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|██████████| 2768/2768 [00:01<00:00, 1606.91it/s]


<H2> 5.1 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [65]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [66]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.1169, recall17: 0.5367, F1-measure: 0.1920


In [67]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

Movie:  Sink the Bismarck!
Actual genre:  romantic, historical
Predicted tag:  ['action' 'alternate history' 'alternate reality' 'anti war'
 'claustrophobic' 'dramatic' 'historical' 'historical fiction' 'intrigue'
 'melodrama' 'sci-fi' 'suspenseful' 'violence'] 

Movie:  Before I Self Destruct
Actual genre:  violence
Predicted tag:  ['bleak' 'clever' 'comedy' 'cute' 'depressing' 'dramatic' 'home movie'
 'intrigue' 'melodrama' 'neo noir' 'plot twist' 'pornographic' 'romantic'
 'sentimental' 'storytelling' 'tragedy'] 

Movie:  The Dark Knight Rises
Actual genre:  dark, suspenseful, neo noir, murder, fantasy, cult, violence, atmospheric, flashback, good versus evil, plot twist, psychedelic, revenge
Predicted tag:  ['action' 'atmospheric' 'bleak' 'brainwashing' 'claustrophobic' 'clever'
 'comedy' 'cult' 'dark' 'good versus evil' 'gothic' 'grindhouse film'
 'horror' 'humor' 'insanity' 'murder' 'mystery' 'neo noir' 'plot twist'
 'realism' 'revenge' 'sadist' 'stupid' 'suspenseful' 'thought-pr

<h2> 5.2 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [68]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [69]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 2*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.0919, recall18: 0.5825, F1-measure: 0.1588


In [70]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

Movie:  La cérémonie
Actual genre:  pornographic, sadist
Predicted tag:  ['absurd' 'alternate history' 'atmospheric' 'autobiographical'
 'avant garde' 'bleak' 'brainwashing' 'claustrophobic' 'clever' 'comedy'
 'cult' 'cute' 'depressing' 'home movie' 'murder' 'mystery' 'neo noir'
 'sadist' 'satire' 'storytelling' 'stupid'] 

Movie:  Western Union
Actual genre:  revenge, romantic
Predicted tag:  ['avant garde' 'depressing' 'historical' 'historical fiction' 'sadist'
 'violence' 'western'] 

Movie:  Trade
Actual genre:  cruelty, murder, violence, flashback, romantic, revenge
Predicted tag:  ['allegory' 'atmospheric' 'avant garde' 'bleak' 'brainwashing'
 'claustrophobic' 'comedy' 'comic' 'depressing' 'dramatic' 'entertaining'
 'fantasy' 'flashback' 'gothic' 'haunting' 'home movie' 'insanity'
 'murder' 'mystery' 'neo noir' 'paranormal' 'plot twist' 'psychological'
 'realism' 'sadist' 'storytelling' 'stupid' 'suicidal' 'suspenseful'
 'thought-provoking' 'violence'] 

Movie:  The Boy Who Could

<h2> 5.3 OneVsRestClassifier + LogisticRegression:</h2>

In [71]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [72]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.1286, recall19: 0.6295, F1-measure: 0.2135


In [73]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

Movie:  Ferdinand the Bull
Actual genre:  psychedelic
Predicted tag:  ['atmospheric' 'autobiographical' 'avant garde' 'bleak' 'comic' 'dramatic'
 'fantasy' 'feel-good' 'good versus evil' 'gothic' 'historical'
 'historical fiction' 'home movie' 'horror' 'inspiring' 'magical realism'
 'prank' 'psychedelic' 'queer' 'suicidal' 'whimsical'] 

Movie:  Junior
Actual genre:  entertaining
Predicted tag:  ['adult comedy' 'alternate reality' 'atmospheric' 'bleak' 'boring'
 'comedy' 'cruelty' 'cult' 'dark' 'dramatic' 'flashback' 'gothic'
 'historical' 'horror' 'insanity' 'magical realism' 'murder' 'neo noir'
 'philosophical' 'pornographic' 'psychedelic' 'psychological' 'queer'
 'sadist' 'satire' 'sentimental' 'storytelling' 'suspenseful'
 'thought-provoking' 'tragedy' 'violence'] 

Movie:  Criminally Insane 2
Actual genre:  murder
Predicted tag:  ['bleak' 'cruelty' 'cute' 'gothic' 'horror' 'insanity' 'magical realism'
 'murder' 'psychedelic' 'realism' 'suspenseful' 'tragedy'] 

Movie:  Le goût des

<h1>Conclusion</h1>

In [74]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision18, 3),  round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |   0.117   | 0.537  |  0.192   |
| SGDClassifier(hinge) |  AVG W2V   |   0.092   | 0.582  |  0.159   |
|  LogisticRegression  |  AVG W2V   |   0.129   | 0.629  |  0.214   |
+----------------------+------------+-----------+--------+----------+


__1.We Did not get the good result using AVGW2V__

<h1>6. LSTM-CNN Model</h1>

In [0]:
max_review_length = 400
X_train = sequence.pad_sequences(X_train_multilabel, maxlen=max_review_length, padding='post')
X_test = sequence.pad_sequences(X_test_multilabel, maxlen=max_review_length, padding='post')

In [0]:
inputt = 8252
batch_size = 32
epochs = 10

In [77]:
model = Sequential()

model.add(Embedding(inputt, 50, input_length=max_review_length))

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(GlobalMaxPooling1D())

model.add(Dense(71, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           412600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 400, 100)          60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 100)          400       
_________________________________________________________________
lstm_2 (LSTM)                (None, 400, 100)          80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 100)         

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [79]:
model.fit(X_train, y_train_multilabel, 
          batch_size = batch_size,
          validation_data=(X_test, y_test_multilabel),
          epochs=epochs)

Train on 10989 samples, validate on 2768 samples
Epoch 1/10
10989/10989 [==============================] - 550s 50ms/step - loss: 10.9269 - accuracy: 0.1259 - val_loss: 11.6958 - val_accuracy: 0.1358
Epoch 2/10
10989/10989 [==============================] - 545s 50ms/step - loss: 10.6147 - accuracy: 0.1421 - val_loss: 11.5913 - val_accuracy: 0.1358
Epoch 3/10
10989/10989 [==============================] - 544s 50ms/step - loss: 10.5880 - accuracy: 0.1421 - val_loss: 11.3198 - val_accuracy: 0.1358
Epoch 4/10
10989/10989 [==============================] - 544s 49ms/step - loss: 10.5796 - accuracy: 0.1421 - val_loss: 11.2531 - val_accuracy: 0.1358
Epoch 5/10
10989/10989 [==============================] - 541s 49ms/step - loss: 10.5758 - accuracy: 0.1421 - val_loss: 11.2143 - val_accuracy: 0.1358
Epoch 6/10
10989/10989 [==============================] - 540s 49ms/step - loss: 10.5703 - accuracy: 0.1421 - val_loss: 11.2298 - val_accuracy: 0.1358
Epoch 7/10
10989/10989 [=====================

In [80]:
test_loss, test_acc = model.evaluate(X_test, y_test_multilabel, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.13583815097808838


In [0]:
model.save('lstm_model_all.h5') #Saving the Model for Future Use

In [0]:
model = load_model('lstm_model_all.h5') #Loading the Model

In [0]:
model_prediction = model.predict(X_test, verbose=0)

In [84]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ", y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(model_prediction[k])[0],"\n")

Movie:  Scarlet Street
Actual genre:  murder
Predicted tag:  ['absurd' 'action' 'adult comedy' 'allegory' 'alternate history'
 'alternate reality' 'anti war' 'atmospheric' 'autobiographical'
 'avant garde' 'blaxploitation' 'bleak' 'boring' 'brainwashing'
 'christian film' 'claustrophobic' 'clever' 'comedy' 'comic' 'cruelty'
 'cult' 'cute' 'dark' 'depressing' 'dramatic' 'entertaining' 'fantasy'
 'feel-good' 'flashback' 'good versus evil' 'gothic' 'grindhouse film'
 'haunting' 'historical' 'historical fiction' 'home movie' 'horror'
 'humor' 'insanity' 'inspiring' 'intrigue' 'magical realism' 'melodrama'
 'murder' 'mystery' 'neo noir' 'non fiction' 'paranormal' 'philosophical'
 'plot twist' 'pornographic' 'prank' 'psychedelic' 'psychological' 'queer'
 'realism' 'revenge' 'romantic' 'sadist' 'satire' 'sci-fi' 'sentimental'
 'storytelling' 'stupid' 'suicidal' 'suspenseful' 'thought-provoking'
 'tragedy' 'violence' 'western' 'whimsical'] 

Movie:  Day of the Dead
Actual genre:  comedy, suspe